In [1]:
import pandas as pd
import numpy as np

# 设置显示所有行和列（仅控制台用）
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# 1) 读入数据
df = pd.read_csv("./car_data.csv", na_values=['-'])

# 2) 转换为数值类型
numeric_cols = [
    'Horsepower(hp)',
    'Acceleration 0 to 100 km/h (seconds)',
    'Average_energy_consumption_1 (kWh/100km)',
    'Average_energy_consumption_2_3 (L/100km)'
]
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# 3) 只保留必要数据存在的行（根据能源类型选择消耗数据）
df = df[
    df['Energy_type'].notna() & (
        ((df['Energy_type'] == 1) & df['Average_energy_consumption_1 (kWh/100km)'].notna()) |
        ((df['Energy_type'] != 1) & df['Average_energy_consumption_2_3 (L/100km)'].notna())
    ) &
    df['Horsepower(hp)'].notna() &
    df['Acceleration 0 to 100 km/h (seconds)'].notna()
]

# 4) 定义碳排和能耗价格参数
co2_intensity_elec = 0.408  # kg CO2/kWh
co2_intensity_fuel = 2.31   # kg CO2/L
price_elec = 0.3            # SGD/kWh
price_fuel = 2.6            # SGD/L

# 5) 计算每万公里碳排放和价格
def calc_emission_and_price(row):
    etype = row['Energy_type']
    if etype == 1:
        consumption = row['Average_energy_consumption_1 (kWh/100km)']
        ghg_100km = consumption * co2_intensity_elec
        price = price_elec
    else:
        consumption = row['Average_energy_consumption_2_3 (L/100km)']
        ghg_100km = consumption * co2_intensity_fuel
        price = price_fuel
    ghg_10000km = ghg_100km * 100
    return ghg_10000km, price

df['GHG_per_10000km'], df['Energy_price'] = zip(
    *df.apply(calc_emission_and_price, axis=1)
)

# 6) 计算不同偏好的评分
def car_score(row, beta, gamma, sigma):
    hp = row['Horsepower(hp)']
    accel_time = row['Acceleration 0 to 100 km/h (seconds)']
    ghg_10k = row['GHG_per_10000km']
    price = row['Energy_price']
    performance_factor = hp * (1.0 / accel_time)
    return (performance_factor**beta) * (ghg_10k**(-gamma)) * (price**(-sigma))

df['Score_perf'] = df.apply(lambda r: car_score(r, 0.8, 0.1, 0.1), axis=1)
df['Score_balanced'] = df.apply(lambda r: car_score(r, 0.4, 0.3, 0.3), axis=1)
df['Score_env'] = df.apply(lambda r: car_score(r, 0.2, 0.7, 0.1), axis=1)
df['Score_daily'] = df.apply(lambda r: car_score(r, 0.1, 0.1, 0.8), axis=1)

# 7) 输出所有评分数据
print(df[[ 
    'Company', 'Car_type',
    'Horsepower(hp)', 'Acceleration 0 to 100 km/h (seconds)',
    'GHG_per_10000km', 'Energy_price',
    'Score_perf', 'Score_balanced', 'Score_env', 'Score_daily'
]])

# 8) 保存为新 CSV
df.to_csv("car_data_scored_full.csv", index=False)


                    Company  \
0                     Tesla   
1                     Tesla   
2                     Tesla   
3                     Tesla   
4                     Tesla   
5                     Tesla   
6                     Tesla   
7                     Tesla   
8                     Tesla   
9                     Tesla   
10                    Tesla   
11                    Tesla   
12                    Tesla   
13                    Tesla   
14                    Tesla   
15                    Tesla   
16                    Tesla   
17                    Tesla   
18                    Tesla   
19                    Tesla   
20                    Tesla   
21                    Tesla   
22                    Tesla   
23                    Tesla   
24                    Tesla   
25                    Tesla   
26                    Tesla   
27                    Tesla   
28                    Tesla   
29                    Tesla   
30               Volkswagen   
31      

In [2]:
# 归一化函数
def normalize_column(series):
    return 100 * (series - series.min()) / (series.max() - series.min())

# 归一化评分
score_columns = ['Score_perf', 'Score_balanced', 'Score_env', 'Score_daily']
for col in score_columns:
    df[col + '_norm'] = normalize_column(df[col])

# 导出到CSV
df.to_csv("./car_data_scored_normalized.csv", index=False)
